<a href="https://colab.research.google.com/github/VicDc/VIC_/blob/main/copia_di_summerize_2_mental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Name_MentalHealth.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/... (your Colab Link)
"""

'Name_MentalHealth.ipynb\n\nAutomatically generated by Colaboratory.\n\nOriginal file is located at\n    https://colab.research.google.com/drive/... (your Colab Link)\n'

In [ ]:
# --- Import Libraries ---
!pip install reportlab
!pip install fpdf
!pip install PyPDF2
!pip install transformers
!pip install sentencepiece


import pandas as pd
import csv
from transformers import pipeline
from fpdf import FPDF
import PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c95a4924b66af12bbbd2f0e1522400e09ece86de15af16cbd2fcce9061220fcd
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.0 MB/s eta 0:00:00


In [ ]:
# --- Load the CSV Data ---
#The CSV file is a resource from Kaggle (https://www.kaggle.com/competitions/playground-series-s4e11/models)
csv_path = 'mental_health_conversations2.csv'

# Since your file is semicolon-delimited, specify sep=';'
df = pd.read_csv(
    csv_path,
    sep=';',  # <-- Tells pandas to split on semicolons
    engine='python',
    on_bad_lines='skip'
)

expected_cols = {"Context", "Response"}
if not expected_cols.issubset(df.columns):
    raise ValueError(f"CSV must have columns {expected_cols}, but found {df.columns}")

print("Data loaded successfully. Preview:")
print(df.head())

Data loaded successfully. Preview:
                                             Context  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                            Response  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  


In [ ]:
#SUMMARIZATION MODEL


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
#4 SUMMARIES FOR EACH ROW

summaries = []

for idx, row in df.iterrows():
    text_to_summarize = f"Context: {row['Context']}\nResponse: {row['Response']}"
    summary_output = summarizer(
        text_to_summarize,
        max_length=60,   # Adjust as needed
        min_length=10,   # Adjust as needed
        do_sample=False
    )
    summary_text = summary_output[0]['summary_text']
    summaries.append(summary_text)

# Combine all summaries into one full report string
report_text = "Report Summary of All Contexts and Responses:\n\n"
for i, s in enumerate(summaries):
    report_text += f"Summary {i+1}:\n{s}\n\n"

# 5 REPORT TO A PDF

pdf_filename = "Name_MentalHealth.pdf"

# Using FPDF to create a simple PDF
pdf = FPDF()
pdf.add_page()

# Set font (required before writing text)
pdf.set_font("Arial", size=12)

# We'll split the text into lines that fit within the PDF.
# FPDF doesn't automatically wrap text, so we do a basic split here.
lines = report_text.split("\n")
for line in lines:
    pdf.multi_cell(0, 5, line)  # '0' width means full page width

pdf.output(pdf_filename)

print(f"PDF '{pdf_filename}' has been created with the summarized report.")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


PDF 'Name_MentalHealth.pdf' has been created with the summarized report.


In [ ]:
#6 READ THE PDF BACK AND ANALYZE CONTENT

#read and analyze the PDF content:
reader = PyPDF2.PdfReader(open(pdf_filename, 'rb'))

# Extract all text from the PDF
extracted_text = ""
for page_num in range(len(reader.pages)):
    extracted_text += reader.pages[page_num].extract_text()

print("Extracted Report Text:")
print("-------------------------------------------------------------")
print(extracted_text)
print("-------------------------------------------------------------\n")



Extracted Report Text:
-------------------------------------------------------------
Report Summary of All Contexts and Responses:
Summary 1:
The social context in which a person lives is a big influence in self-esteem. If everyone thinks you're
worthless, then maybe you need to find new people to hang out with.
Summary 2:
I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think
about how I'm worthless and how I shouldn't be here. I've never tried or contemplated suicide, but
I've always wanted to fix my issues. How can I change my
Summary 3:
"I do nothing but think about how I'm worthless and how I shouldn't be here" "I've never tried or
contemplated suicide. I've always wanted to fix my issues, but I never get around to it" "Everyone
has talents and wants to find their purpose in life
Summary 4:
CBT is essential for those that are feeling depressed and worthless. CBT is good for individuals
dealing with depression, anxiety, toxic relationsh

In [ ]:
# 7) SUMMARIZE ALL EXTRACTED TEXT AND PRINT (MAX 500 WORDS)
def chunk_text(text, chunk_size=512):
    """
    Splits 'text' into chunks, each up to 'chunk_size' words.
    You can reduce chunk_size if summarizer still crashes,
    or increase it if you want fewer, bigger chunks.
    """
    words = text.split()
    for i in range(0, len(words), chunk_size):
        yield " ".join(words[i:i + chunk_size])


##########################################
# 9) SUMMARIZE IN CHUNKS, THEN COMBINE
##########################################

# Initialize summarizer on CPU (device=-1) to avoid GPU issues:
final_summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1
)

# 9a) Split the entire extracted text into smaller pieces
chunks = list(chunk_text(extracted_text, chunk_size=512))

partial_summaries = []
for i, c in enumerate(chunks):
    # Summarize each chunk separately
    summary_result = final_summarizer(
        c,
        max_length=150,  # You can adjust as needed
        min_length=40,
        do_sample=False,
        truncation=True
    )
    partial_summary = summary_result[0]['summary_text']
    partial_summaries.append(partial_summary)

# 9b) Combine these partial summaries into one big text
combined_summary = " ".join(partial_summaries)

# 9c) Optionally re-summarize that combined summary to make it even more concise
final_summary_output = final_summarizer(
    combined_summary,
    max_length=200,
    min_length=50,
    do_sample=False,
    truncation=True
)

final_summary = final_summary_output[0]['summary_text']

# 9d) Enforce a max 500-word limit
final_words = final_summary.split()
if len(final_words) > 500:
    final_summary = " ".join(final_words[:500]) + " ..."

# 9e) Add some extra context
report_header = "FINAL SUMMERY REPORT\n\n"
report_info = (
    "Below is the final synthesized summary of all extracted PDF content, "
    "condensed from multiple chunked analyses. It provides a high-level overview "
    "of key points from the original mental health conversations.\n\n"
)
full_final_report = report_header + report_info + final_summary

print("FINAL SUMMARY OF THE PDF CONTENT (<= 500 words):")
print("------------------------------------------------")
print(full_final_report)


##########################################
# 10) SAVE THE FINAL REPORT TO A PDF
##########################################

final_pdf_name = "FINAL_SUMMERY_REPORT.pdf"
pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Simple line-by-line writing
report_lines = full_final_report.split("\n")
for line in report_lines:
    pdf.multi_cell(0, 5, line)

pdf.output(final_pdf_name)

print(f"PDF '{final_pdf_name}' has been created with the final summary report.")


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 200, but your input_length is only 112. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)


FINAL SUMMARY OF THE PDF CONTENT (<= 500 words):
------------------------------------------------
FINAL SUMMERY REPORT

Below is the final synthesized summary of all extracted PDF content, condensed from multiple chunked analyses. It provides a high-level overview of key points from the original mental health conversations.

Social context in which a person lives is a big influence in self-esteem. Visiting your doctor is a great first step to address your lack of sleep. One activity that people have found helpful in feeling better about themselves is keeping a gratitude journal.
PDF 'FINAL_SUMMERY_REPORT.pdf' has been created with the final summary report.


In [ ]:
# 7) SUMMARIZE ALL EXTRACTED TEXT AND PRINT
##########################################
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
# We'll use the same summarizer pipeline to condense the entire PDF content.
final_summary_output = summarizer(
    extracted_text,
    max_length=500,  # You can adjust this limit up or down
    min_length=100,   # Provide a minimum length to avoid extremely short summary
    do_sample=False
)

final_summary = final_summary_output[0]['summary_text']

print("FINAL SUMMARY OF THE PDF CONTENT:")
print("---------------------------------")
print(final_summary)

Device set to use cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
##########################################
# 7) ANALYZE BIAS AND FAIRNESS
##########################################

# For a simple demonstration, we might do a naive check for certain terms or sentiments,
# or use a zero-shot classifier to see if the text has indications of bias.
# Below is a very basic example.

# Example: Using a zero-shot classification approach to detect possible "bias" or "fairness"
# categories in the text. (Note: This is purely illustrative!)
from transformers import pipeline

classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

candidate_labels = ["fairness", "unbiased", "bias", "discrimination"]

analysis = classifier(extracted_text, candidate_labels)
print("Bias/Fairness Analysis (Zero-shot classification example):")
print(analysis)

print("\n--- Analysis Complete ---\n")
print("You may refine or expand this analysis with more rigorous methods or datasets.")